In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iis-fukushima/2016_2020_fukushima.xlsx
/kaggle/input/stop-words-japanese/stop_words_japanese.txt
/kaggle/input/iis-dataset/data(new)/2011_2015_miyagi.xlsx
/kaggle/input/iis-dataset/data(new)/2015_2019_miyagi.xlsx
/kaggle/input/iis-dataset/data(new)/2011_2015_fukushima.xlsx
/kaggle/input/iis-dataset/data(new)/2010_2014_ibaraki.xlsx
/kaggle/input/iis-dataset/data(new)/2015_2019_fukushima.xlsx
/kaggle/input/iis-dataset/data(new)/2015_2019_iwate.xlsx
/kaggle/input/iis-dataset/data(new)/2011_2015_iwate.xlsx


In [4]:
from transformers import logging, pipeline
from transformers import AutoModel, AutoTokenizer, BertModel, BertForMaskedLM#, BertTokenizer AutoModel
from bertopic import BERTopic
from umap import UMAP

import torch

import pandas as pd
import numpy as np
import random
import copy
import re
import pdb

#pdb.set_trace()
# define
# for the topic
umap_model = UMAP(n_neighbors=15, n_components=5,
                  min_dist=0.0, metric='cosine', random_state=42)

model = BERTopic(language="Japanese",
                 embedding_model="paraphrase-multilingual-MiniLM-L12-v2",
                 umap_model=umap_model)
#pdb.set_trace()
# sentiment
sem_model = pipeline("sentiment-analysis",
                     model="koheiduck/bert-japanese-finetuned-sentiment",
                     tokenizer="koheiduck/bert-japanese-finetuned-sentiment")

Bertmodel = AutoModel.from_pretrained("rinna/japanese-roberta-base",
                                      output_hidden_states=True)
# tokenizer
Tokenizer = AutoTokenizer.from_pretrained('rinna/japanese-roberta-base')

word_list = ['G20','Io','20代','30代','10代','40代','50代'] # this is an example
word_list2 = ['IoT','広域での連携'] # this is an example

replace_list = [['県の','県内','本県'],['国際的な','世界の','世界に','▁世界','の世界','国際','海外']] # this is an example
cp_replace_list = ['県','世界','日本','貿易'] # this is an example

stopword_list=[k.strip() for k in open('/kaggle/input/stop-words-japanese/stop_words_japanese.txt',encoding='utf8').readlines() if k.strip()!='']
add_sw = ['も','の','い','で','ど','この','と','ます','を','が','お','は', 'いた','き', 'ました','き','や',
          '1','2','3','4','5','6','7','8','9','10','21','24','60','70','%',
         ',','.','▁', '◯','，','。','！','？','：','《','》','“','”','’','‘','「','」','(',')','・',] # this is an example
stopword_list.extend(add_sw)

# random seed
def set_seed(seed=2021):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()
def read_splitwords():
    splitword_list=[]#[k.strip() for k in open('/kaggle/input/splitw/split.txt',encoding='utf8').readlines() if k.strip()!='']
    splitword_list = ['\n'+i.split('.')[-1] for i in splitword_list if '/' not in i and '（前边没有空格)' not in i]
    splitword_list2 = ['\n\u3000'+i.split('.')[-1] for i in splitword_list if '/' not in i and '（前边没有空格)' not in i]
    splitword_list.extend(splitword_list2)
    splitword_list = '|'.join(i for i in splitword_list)
    return splitword_list

# split paragraphes and get the time list
def dynamic_time_text(df, row, col, target_col='change_level',foreachpeople=False):
    test_text, times_list, tem_time_min = df.loc[row, col], [], 0
    splitword_list = read_splitwords()
    textn = re.split(splitword_list, test_text)
    target_list = [df.loc[row, target_col] for i in range(len(textn))]
    test_text_list = [len(textn[i]) / len(test_text) for i in range(len(textn))]
    if foreachpeople:
        for i in range(len(test_text)):
            tem_time_min += round(len(test_text) / len(test_text) * 59)
            temp_time = '{}-{}'.format(col.split('.')[0], col.split('.')[1], str(0).zfill(2))
            times_list.append(temp_time)
    else:
        for i in range(len(textn)):
            #print ('for whole')
            tem_time_min += round(len(textn[i]) / len(test_text) * 59)
            #tem_time_min = tem_time_min -1 if i == len(textn)-1 else tem_time_min
            tem_time_min = 59 if tem_time_min >59.5 else tem_time_min
            tem_time_min = 0 if tem_time_min <0 else tem_time_min
            temp_time = '{}-{}-01 10:{}:00'.format(col.split('.')[0], col.split('.')[1], str(tem_time_min).zfill(2))
            times_list.append(temp_time)
    return times_list, textn, target_list, test_text_list, [len(textn)]

def dynamic_time_text1(df, row, col, target_col='change_level',foreachpeople=True):
    test_text, times_list, tem_time_min = df.loc[row, col], [], 0
    splitword_list = read_splitwords()
    textn = re.split(splitword_list, test_text)
    target_list = [df.loc[row, target_col] for i in range(len(textn))]
    test_text_list = [len(textn[i]) / len(test_text) for i in range(len(textn))]
    if foreachpeople:
        for i in range(len(textn)):
            #print ('for whole')
            tem_time_min += round(len(textn[i]) / len(test_text) * 59)
            tem_time_min = 59 if tem_time_min >59.5 else tem_time_min
            tem_time_min = 0 if tem_time_min <0 else tem_time_min
            temp_time = '{}-{}-01 10:{}:00'.format(col.split('.')[0], col.split('.')[1], str(tem_time_min).zfill(2))
            times_list.append(temp_time)
    return times_list, textn, target_list, test_text_list, [len(textn)]

def get_time_text(df, col_time,foreachpeoplea=True):
    row_col, time, text, target_l, text_len, textnumber, subnumber = [], [], [], [], [], [], []
    for i in range(0, len(df)):
        col_time_each = [j for j in col_time if pd.isnull(df.loc[i, j]) == False]
        row_col.append([i, col_time_each]) if col_time_each else row_col
    for j in range(len(row_col)):
        
        for i in row_col[j][1]:
            
            a, b, c, d, e = dynamic_time_text(df, row_col[j][0], i, foreachpeople=foreachpeoplea)  # all text
            time.extend(a) #[i]
            text.extend(b)
            target_l.extend([c[0]])
            text_len.extend(d)
            textnumber.extend(e)
        
        subnumber.extend([len(row_col[j][1])])
    return time, text, target_l, text_len, textnumber, subnumber

def get_two_kinds_text0(tem_text,
                       tem_tokenizer=AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')):
    tem_text_in_list = copy.deepcopy(tem_text)
    for j in range(len(tem_text)):
        tem_text_in_list[j] = tem_tokenizer.tokenize(tem_text_in_list[j])  # all tokens in list
        tem_text[j] = ' '.join(i for i in tem_tokenizer.tokenize(tem_text[j])).replace('#', '')  # all tokens in string
    return tem_text_in_list, tem_text

def get_two_kinds_text(tem_text,
                       tem_tokenizer=AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')):
    tem_text_in_list = copy.deepcopy(tem_text)
    for j in range(len(tem_text)):
        tem_text_in_list[j] = tem_tokenizer.tokenize(tem_text_in_list[j])  # all tokens in list
        tem_text_in_list[j] = comb_rep([tem_text_in_list[j]],word_list,word_list2,replace_list,cp_replace_list)
        tem_text_in_list[j] = use_stopwords([tem_text_in_list[j][0]],stopword_list)
        tem_text[j] = ' '.join(i for i in tem_text_in_list[j][0])   # all tokens in string
    print ('clean!')
    print ('removed stop words!')
    return tem_text_in_list, tem_text





# topic
def topic_each_paragraph(tem_text,tem_model = model):
    tem_topics, tem_probabilities = tem_model.fit_transform(tem_text)
    tem_freq = tem_model.get_topic_info()
    tem_topic = tem_freq.loc[0, 'Name'] if tem_freq.loc[0,'Topic'] != -1 or len(tem_freq) == 1 else tem_freq.loc[1, 'Name']
    return tem_topic.replace('0_', '').replace('1_', '').replace('-1_', '').replace('_', ';')

# sentiment
def each_sentiment(tem_sem_text,
                   tem_sem_model = sem_model):
    tem_sentiment = tem_sem_model(tem_sem_text)
    return [tem_sentiment[0]['label'],tem_sentiment[0]['score']]

# embedding
def bert_embedding(rep_docs,
                   bertmodel = Bertmodel,
                   four_layer_embedding = True,
                   tem_tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')):
    granola_ids = torch.LongTensor(tem_tokenizer.encode(rep_docs)).unsqueeze(0)
    hidden_states = bertmodel(input_ids=granola_ids)[2] # tuple
    if four_layer_embedding:
        last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)] # get last four layers
        cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1) # cast layers to a tuple and concatenate over the last dimension
        sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze() # take the mean of the concatenated vector over the token dimension
    else:
        sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze() #; print(sentence_embedding.size())
    return sentence_embedding

# post
def texttime(ID):
    tmp_textt = text2[rlist[ID][2]:rlist[ID][3]]
    
    tmp_timec = []
    for i in range(rlist[ID][0],rlist[ID][1]):
        for k in range(textnumber[i]):
            tmp_timec.append(time[i])
    return tmp_textt, tmp_timec

from tqdm.notebook import tqdm
def topicmodel(textt, timec):
    topics, probabilities = model.fit_transform(textt)
    topics_over_time = model.topics_over_time(docs=text, 
                                                    topics=topics, 
                                                    timestamps=timec, 
                                                    global_tuning=True, 
                                                    evolution_tuning=True, 
                                                    )
    topic_df = model.get_topic_info().head(10)
    topic_df_words = pd.DataFrame(model.get_topic(0)) if len(model.get_topic_info().head(10)) != 1 else pd.DataFrame()
    return pd.concat([topic_df.reset_index(drop= True), topic_df_words.reset_index(drop= True)],axis=1)


# words
def use_stopwords(tem_listss,tem_stopword_list = stopword_list):
    for i in tem_stopword_list:
        for k in tem_listss:
            while i in k:
                k.remove(i)
    return tem_listss

def add_words(spr_list, word):
    r_list = []
    for i in range(len(spr_list)-1):
        if spr_list[i]+spr_list[i+1] == word:
            spr_list[i] = word
            r_list.append(i+1)
    count = 0
    for k in r_list:
        #print (k)
        spr_list.pop(k-count)
        count += 1
    return spr_list

def comb_rep(tem_text_in_list,word_list,word_list2,replace_list,cp_replace_list):
    for seq in range(len(tem_text_in_list)):
        # 合并词
        list_f = tem_text_in_list[seq].copy()
        for j in word_list:
            list_f2 = add_words(list_f, j)
        for l in word_list2:
            tem_text_in_list[seq] = add_words(list_f2, l)

        # 替换同义词
        for j in range(len(cp_replace_list)):
            tem_text_in_list[seq] = [cp_replace_list[j] if i in replace_list[j] else i for i in tem_text_in_list[seq]]
    return tem_text_in_list

def preprocess_features_wo(Path, savetxt = False, rangea = 0, rangeb = 4):
    df = pd.read_excel(Path)
    df.columns = [str(i) for i in df.columns]
    col_time = [i for i in df.columns if re.match(r'\d{4}.\d{1}|\d{4}.\d{2}',i)]

    time, text, target, textlen = get_time_text(df, col_time); print ('got target, time and text!')
    text_in_list, text = get_two_kinds_text(text, tem_tokenizer = Tokenizer); print ('got 2 kinds of text!')
    text_in_list = use_stopwords(text_in_list,stopword_list); print ('removed stop words!')
    text_in_list = comb_rep(text_in_list,word_list,word_list2,replace_list,cp_replace_list);print ('clean!')
    embedding_list = []

    for num, i in enumerate(tqdm(text_in_list[rangea:rangeb])):
        #gc.collect()
        if len(i) > 18:
            representative_docs = topic_each_paragraph(i)#; print ('got the first topic!')
            topic_embedding = bert_embedding(representative_docs, four_layer_embedding=False,
                                             tem_tokenizer=Tokenizer)#; print ('got the topic embedding!')
            sentiment_clf, sent_prob = each_sentiment(i)#; print ('got the sentiment!')
            sentiment_embedding = bert_embedding(sentiment_clf, four_layer_embedding=False,
                                                 tem_tokenizer=Tokenizer) * sent_prob#; print ('got the sentiment embedding!')
            time_embedding = bert_embedding(time[num], four_layer_embedding=False,
                                            tem_tokenizer=Tokenizer)#; print ('got the time embedding!')
            passage_embedding = (topic_embedding + sentiment_embedding) * textlen[
                num] + time_embedding#; print ('got the final embedding!')
        '''
        else:
            text_in_list[num] = ['意味がない']
            passage_embedding = bert_embedding('意味がない',
                                               four_layer_embedding=False, tem_tokenizer=Tokenizer)
        '''
        embedding_list.append(passage_embedding.detach().numpy())
    data = dict({'time':time_embedding,
                 'topic': topic_embedding,
                 'semantic':sentiment_embedding,
                 'label':target,
                  })
    if savetxt:
        np.savetxt('./output/1_time_{}.csv'.format(Path.split('/')[-1].split('.')[0]), time[rangea:rangeb], fmt='%s', delimiter=',')
        np.savetxt('./output/2_text_{}.csv'.format(Path.split('/')[-1].split('.')[0]), text[rangea:rangeb], fmt='%s', delimiter=',')
        np.savetxt('./output/3_target_{}.csv'.format(Path.split('/')[-1].split('.')[0]), target[rangea:rangeb], fmt='%s', delimiter=',')
        np.savetxt('./output/4_embedding_{}.csv'.format(Path.split('/')[-1].split('.')[0]), embedding_list, fmt='%s', delimiter=',')
    return data#time, text, target, embedding_list

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Some weights of the model checkpoint at rinna/japanese-roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
d2010ibaraki = False
d2014ibaraki = False
d2011miyagi = False#True
d2015miyagi = False#True
d2011iwate = False#True
d2015iwate = False#True
d2011fukushima = False#True
d2015fukushima = False#True
d2016fukushima = True#False


if d2010ibaraki:
    print ('data:2010-2014ibaraki')
    df = pd.read_excel('/kaggle/input/march18/2010_2014_ibaraki.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2010.3', '2010.6', '2010.9', '2010.12', '2011.3', '2011.6', '2011.9',
           '2011.12', '2012.3', '2012.6', '2012.9', '2012.12', '2013.3', '2013.6',
           '2013.9', '2013.12', '2014.3', '2014.6', '2014.9', '2014.12']
    
    
if d2014ibaraki:
    print ('data: 2014-2018ibaraki')
    df = pd.read_excel('/kaggle/input/march18/2014_2018_ibaraki.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2014.2', '2014.6', '2014.9', '2014.12', '2015.2', '2015.6', '2015.9',
       '2015.12', '2016.2', '2016.6', '2016.9', '2016.12', '2017.2', '2017.6',
       '2017.9', '2017.12', '2018.2', '2018.6', '2018.9', '2018.12']
    

if d2011miyagi:
    print ('data: 2011-2015miyagi')
    df = pd.read_excel('/kaggle/input/march18/2011_2015_miyagi.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2011.2', '2011.6', '2011.9', '2011.12', '2012.2', '2012.6', '2012.9',
       '2012.12', '2013.2', '2013.6', '2013.9', '2013.12', '2014.2', '2014.6',
       '2014.9', '2014.12', '2015.2', '2015.6', '2015.9', '2015.12']

if d2015miyagi:
    print ('data: 2015-2019miyagi')
    df = pd.read_excel('/kaggle/input/march18/2015_2019_miyagi.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2015.2', '2015.6', '2015.9', '2015.12', '2016.2', '2016.6', '2016.9',
       '2016.12', '2017.2', '2017.6', '2017.9', '2017.12', '2018.2', '2018.6',
       '2018.9', '2018.12', '2019.2', '2019.6', '2019.9', '2019.12']
    
if d2011iwate:
    print ('data: 2011-2015iwate')
    df = pd.read_excel('/kaggle/input/march18/2011_2015_iwate.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2011.2', '2011.6', '2011.9', '2011.12', '2012.2', '2012.6', '2012.9',
       '2012.12', '2013.2', '2013.6', '2013.9', '2013.12', '2014.2', '2014.6',
       '2014.9', '2014.12', '2015.2', '2015.6', '2015.9', '2015.12']

if d2015iwate:
    print ('data: 2015-2019iwate')
    df = pd.read_excel('/kaggle/input/march18/2015_2019_iwate.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2015.2', '2015.6', '2015.9', '2015.12', '2016.2', '2016.6', '2016.9',
       '2016.12', '2017.2', '2017.6', '2017.9', '2017.12', '2018.2', '2018.6',
       '2018.9', '2018.12', '2019.2', '2019.6', '2019.9', '2019.12']
    
if d2011fukushima:
    print ('data: 2011-2015fukushima')
    df = pd.read_excel('/kaggle/input/iis-dataset/data(new)/2011_2015_fukushima.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2011.2', '2011.6', '2011.9', '2011.12', '2012.2', '2012.6', '2012.9',
       '2012.12', '2013.2', '2013.6', '2013.9', '2013.12', '2014.2', '2014.6',
       '2014.9', '2014.12', '2015.2', '2015.6', '2015.9', '2015.12']

if d2015fukushima:
    print ('data: 2015-2019fukushima')
    df = pd.read_excel('/kaggle/input/march18/2015_2019_iwate.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2015.2', '2015.6', '2015.9', '2015.12', '2016.2', '2016.6', '2016.9',
       '2016.12', '2017.2', '2017.6', '2017.9', '2017.12', '2018.2', '2018.6',
       '2018.9', '2018.12', '2019.2', '2019.6', '2019.9', '2019.12']
    
if d2016fukushima:
    print ('data: 2016-2020fukushima')
    df = pd.read_excel('/kaggle/input/iis-fukushima/2016_2020_fukushima.xlsx')
    df.columns = [str(i) for i in df.columns]
    col_time = ['2016.2', '2016.6', '2016.9', '2016.12', '2017.2', '2017.6', '2017.9',
       '2017.12', '2018.2', '2018.6', '2018.9', '2018.12', '2019.2', '2019.6',
       '2019.9', '2019.12', '2020.2', '2020.6', '2020.9', '2020.12']


data: 2016-2020fukushima


In [6]:
time, text, target, textlen, textnumber, subnumber = get_time_text(df, 
                                                                   col_time,
                                                                   foreachpeoplea=False); print ('got target, time and text!')
text_in_list, text2 = get_two_kinds_text(text, tem_tokenizer = Tokenizer); print ('got 2 kinds of text!')

print (len(text2))

got target, time and text!
clean!
removed stop words!
got 2 kinds of text!
1318


In [8]:
topics, probabilities = model.fit_transform(text2)
topics_over_time = model.topics_over_time(docs=text2, 
                                                topics=topics, 
                                                timestamps=time, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                )

In [9]:
%matplotlib inline
fig2=model.visualize_topics_over_time(topics_over_time, 
                                 top_n_topics=5,
                                normalize_frequency=True) #,normalize_frequency
fig2.show()

In [12]:
model.get_topic_freq().head()

,Topic,Count
0,-1,414
1,0,117
2,1,77
3,2,71
4,3,66


In [14]:
model.get_topic_info().head(35)

,Topic,Count,Name
0,-1,414,-1_振興_子ども_日本_制度
1,0,117,0_医療_がん_医師_高齢
2,1,77,1_災害_台風_被害_水害
3,2,71,2_農業_農家_農地_gap
4,3,66,3_道路_交通_運転_自転車
5,4,54,4_通告_連合_質問_に従い
6,5,46,5_学校_教育_高校_県立
7,6,40,6_人口_地区_減少_対策
8,7,38,7_知事_振興_住民_質問
9,8,34,8_ロボット_創造_構想_産業


In [15]:
fig2=model.visualize_barchart(top_n_topics=50, height=200)
fig2.show()

In [16]:
model.visualize_term_rank(log_scale=True)

In [17]:
fig2 = model.visualize_hierarchy(top_n_topics=50, width=1000)

fig2.show()
fig2.write_html("file.html")

In [18]:
model.visualize_hierarchy(top_n_topics=50, width=800)